In [1]:
import pandas as pd

# Read and concatenate the data files
file_paths = ['/kaggle/input/netflix-prize-data/combined_data_1.txt'
              , '/kaggle/input/netflix-prize-data/combined_data_2.txt'
              ,'/kaggle/input/netflix-prize-data/combined_data_3.txt'
              ,'/kaggle/input/netflix-prize-data/combined_data_4.txt'
             ]

dfs = []
for file_path in file_paths:
    df = pd.read_csv(file_path, header=None, names=['userId', 'rating'], usecols=[0, 1])
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

# Identify indices of NaN values in 'rating' column
nan_indices = df[df['rating'].isna()].index.to_list()

# Extract ratings data
ratings = []
for i in range(len(nan_indices)):
    start_index = nan_indices[i] + 1
    end_index = nan_indices[i + 1] - 1 if i + 1 < len(nan_indices) else None
    temp_df = df.loc[start_index:end_index].reset_index(drop=True)
    temp_df['movieId'] = df.loc[nan_indices[i], 'userId'][:-1]
    ratings.append(temp_df)

df = pd.concat(ratings, ignore_index=True)
del ratings, dfs
df.head()

,userId,rating,movieId
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1


In [2]:
top_users = df['userId'].value_counts(normalize=True).reset_index()
print('top users proportion of total ratings : ', top_users[:65000].userId.sum()*100)
top_users = top_users[:65000]['index']

top_movies = df['movieId'].value_counts(normalize=True).reset_index()
print('top movies proportion of total ratings : ',top_movies[:1800].movieId.sum())
top_movies = top_movies[:1800]['index']

top users proportion of total ratings :  51.81983307468781
top movies proportion of total ratings :  0.7688015447613138


In [3]:
print(len(df))
df = df[df['userId'].isin(top_users.to_list())]
df = df[df['movieId'].isin(top_movies.to_list())]
df.reset_index(drop=True, inplace=True)
del top_movies, top_users 
print(len(df))

100480507
37193851


In [4]:
df.to_csv("/kaggle/working/netflix_interactions_lite.csv")